### 목록
* 준비
    * [사용할 라이브러리 불러오기](#lib)
    * [엘라스틱서치 환경 설정하기](#es_setting)
    * [엘라스틱서치에서 데이터 조회하기](#es_load)
    * [조회한 데이터를 테이블 형태로 만들기](#df)
* Parent Pipeline Aggregation
    * [Derivative Aggregation](#der)
    * [Cumulative Sum Aggregation](#cusum)
    * [Moving Avg Aggregation](#mvavg)
    * [Serial Diff Aggregation](#sd)
---

### 준비

<a name='lib'></a>
#### 사용할 라이브러리 불러오기

In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
import datetime

<a name='es_setting'></a>
#### 엘라스틱서치 환경 설정하기

In [2]:
# 적당한 값을 넣지 않으면 오류가 납니다
es = Elasticsearch()
es_index = 'test'
es_type = 'test'

<a name='es_load'></a>
#### 엘라스틱서치에서 데이터 조회하기

In [3]:
x = es.search(
      index=es_index, 
      doc_type=es_type, 
      body={
        '_source' : ['주문시간', '상품분류', '고객나이', '구매사이트'],
        'query' : {
            'match_all' : {}
        },
        'size' : 50
      }
)

<a name='df'></a>
#### 조회한 데이터를 테이블 형태로 만들기

In [4]:
df = pd.DataFrame([x['hits']['hits'][idx]['_source'] for idx, _ in enumerate(x['hits']['hits'])])
df

,고객나이,구매사이트,상품분류,주문시간
0,36,g마켓,스커트,2017-11-04T04:39:24
1,28,옥션,팬츠,2017-11-07T02:27:49
2,45,11번가,스웨터,2017-11-01T21:10:05
3,40,쿠팡,티셔츠,2017-11-12T11:30:19
4,40,11번가,코트,2017-11-12T18:11:58
5,38,11번가,니트,2017-11-08T22:29:44
6,45,위메프,니트,2017-11-07T01:22:52
7,20,위메프,스웨터,2017-11-11T06:25:42
8,34,g마켓,가디건,2017-11-09T01:14:04
9,33,g마켓,가디건,2017-11-02T20:24:25


### Parent Piepline Aggregation

In [5]:
df.index = pd.to_datetime(df['주문시간'])
df_daily_count = df.resample('D').count().iloc[:, 0]
df_daily_count_ = pd.DataFrame({'주문시간' : df_daily_count.index, '개수' : df_daily_count.values.ravel()})
df_daily_count_[['주문시간', '개수']]

,주문시간,개수
0,2017-11-01,5
1,2017-11-02,4
2,2017-11-03,3
3,2017-11-04,3
4,2017-11-05,1
5,2017-11-06,3
6,2017-11-07,6
7,2017-11-08,5
8,2017-11-09,7
9,2017-11-10,3


<a name="der"></a>
#### Derivative Aggregation 

: (Date) Histogram에서 연속된 두 bucket의 aggregated된 값의 차이를 구한다. (현재 값 - 이전 값)

In [6]:
df_daily_count_diff = df_daily_count.diff()
df_derivative = pd.concat([df_daily_count, df_daily_count_diff], axis=1)
df_derivative.columns.values[0], df_derivative.columns.values[1]  = '일별 판매 개수', '전일 대비 판매 개수 변화'
df_derivative

,일별 판매 개수,전일 대비 판매 개수 변화
주문시간,,
2017-11-01,5,NaN
2017-11-02,4,-1.0
2017-11-03,3,-1.0
2017-11-04,3,0.0
2017-11-05,1,-2.0
2017-11-06,3,2.0
2017-11-07,6,3.0
2017-11-08,5,-1.0
2017-11-09,7,2.0


<a name="cusum"></a>

#### Cumulative Sum Aggregation
: (Date) Histogram에서 bucket의 aggregate된 값의 누적합 반환한다

In [7]:
df_daily_count_cumsum = df_daily_count.cumsum()
df_cumsum = pd.concat([df_daily_count, df_daily_count_cumsum], axis=1)
df_cumsum.columns.values[0], df_cumsum.columns.values[1]  = '일별 판매 개수', '누적 판매 개수'
df_cumsum

,일별 판매 개수,누적 판매 개수
주문시간,,
2017-11-01,5,5
2017-11-02,4,9
2017-11-03,3,12
2017-11-04,3,15
2017-11-05,1,16
2017-11-06,3,19
2017-11-07,6,25
2017-11-08,5,30
2017-11-09,7,37


<a name="mvavg"></a>
#### Moving Avg Aggregation

: (Date) Histogram에서 bucket의 aggregate된 값들의 이동평균 반환한다

In [8]:
# Kibana와 값이 살짝 다른데 흐름만 보고 넘어가주세요!
df_daily_count_rolling = df_daily_count.rolling(window=5, closed='left', min_periods=1).mean()
df_rolling = pd.concat([df_daily_count, df_daily_count_rolling], axis=1)
df_rolling.columns.values[0], df_rolling.columns.values[1]  = '일별 판매 개수', '이틀 간 판매 개수의 이동 평균'
df_rolling

,일별 판매 개수,이틀 간 판매 개수의 이동 평균
주문시간,,
2017-11-01,5,5.00
2017-11-02,4,4.50
2017-11-03,3,4.00
2017-11-04,3,3.75
2017-11-05,1,3.20
2017-11-06,3,2.80
2017-11-07,6,3.20
2017-11-08,5,3.60
2017-11-09,7,4.40


<a name="sd"></a>
#### Serial Diff Aggregation 

: (Date) Histogram에서 현재 bucket의 aggregate된 값과 이전 {n}번째 bucket의 aggregate된 값의 차이를 반환한다 (현재 - 이전)

In [9]:
df_daily_count_diff_ = df_daily_count.diff(periods=1)
df_derivative_ = pd.concat([df_daily_count, df_daily_count_diff_], axis=1)
df_derivative_.columns.values[0], df_derivative_.columns.values[1]  = '일별 판매 개수', '전전일 대비 판매 개수 변화'
df_derivative_

,일별 판매 개수,전전일 대비 판매 개수 변화
주문시간,,
2017-11-01,5,NaN
2017-11-02,4,-1.0
2017-11-03,3,-1.0
2017-11-04,3,0.0
2017-11-05,1,-2.0
2017-11-06,3,2.0
2017-11-07,6,3.0
2017-11-08,5,-1.0
2017-11-09,7,2.0
